<h1 style="text-align: center;">The 'physics' package</h1>

This notebook shows how to use *femtoscope* `physics` package to avoid having to define the weak form 'by hand'.

*prerequisites* :
- FEM knowledge
- `mesh_generation_basics` notebook

**If you have questions/comments or want to report a bug, please send me an email at <a href="mailto:hugo.levy@onera.fr">hugo.levy@onera.fr</a>**

In [1]:
# Add femtoscope to the path
%reset
%matplotlib inline
import sys
sys.path.append("../") # go to parent dir

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


## `femtoscope.physics`

The package `femtoscope.physics` contains two modules:
- `material_library`. It is a library of material functions, sorted by type (laplace term, density term, nonlinear term, etc.), accessible by the dimension and a coordinate system.
- `physical_problems`. Module where weak forms of recurrent problems are registered. All the material functions are accessed from the `material_library`. The currently implemented physical problems are the Poisson problem (class `Poisson`) and the Chameleon field equation (class `Chameleon`). They are in charge of constructing the relevant weak forms (`wf_int`, `wf_ext` for solving the PDE on an unbounded domain, `wf_residual` when the problem at stake is nonlinear), setting a default solver (`LinearSolver` or `NonLinearSolver` instance) and a default monitor (`NonLinearMonitor` instance when the problem at stake is nonlinear). This is where new physical problems are to be defined.

In this notebook, we showcase the `physical_problems` module.

## Poisson problem on unbounded domain made easy

Here, we compute the gravitational potential created by an oblate spheroid

In [2]:
# Module imports
import numpy as np
from numpy import pi
from femtoscope.physics.physical_problems import Poisson, Chameleon
from femtoscope.inout.meshfactory import MeshingTools

install jax and jaxlib to use terms_jax.py!
sfepy: WARNING: module C:\Anaconda3\envs\phd\Lib\site-packages\sfepy\terms\terms_jax.py cannot be imported!
sfepy: reason:
 cannot import name 'config' from 'jax.config' (C:\Anaconda3\envs\phd\lib\site-packages\jax\config.py)


In [5]:
# parameters
sa = 1.2
sc = 0.8
Rc = 5.0
alpha = 4*pi
meshint_name = 'mesh_sphere_int.vtk'
meshext_name = 'mesh_sphere_ext.vtk'
fem_order = 2
dim = 2
coorsys = 'cylindrical'

# Meshes
def get_pre_meshes():
    return mesh2dcyl_int(), mesh2dcyl_ext()

def mesh2dcyl_int():
    mt = MeshingTools(2)
    s1 = mt.create_ellipse(rx=sa, ry=sc)
    mt.create_subdomain(cell_size_min=0.05, cell_size_max=0.2, dist_min=0.0, dist_max=4.0)
    s2 = mt.create_disk_from_pts(Rc, N=200)
    mt.subtract_shapes(s2, s1, removeObject=True, removeTool=False)
    mt.create_subdomain(cell_size_min=0.2, cell_size_max=0.2)
    return mt.generate_mesh(meshint_name, cylindrical_symmetry=True, show_mesh=True, ignored_tags=[200])

def mesh2dcyl_ext():
    mt = MeshingTools(2)
    mt.create_disk_from_pts(Rc, N=200)
    mt.create_subdomain(cell_size_min=0.2, cell_size_max=0.2)
    origin_rf = [0.07, 0.2, 0.1, 3.0]
    return mt.generate_mesh(meshext_name, cylindrical_symmetry=True, show_mesh=True, embed_origin=True, origin_rf=origin_rf)

# Creation and setting of the 'Poisson' object
poisson = Poisson({'alpha': alpha}, dim, Rc=Rc, coorsys=coorsys)
pre_mesh_int, pre_mesh_ext = get_pre_meshes()
partial_args_dict_int = {'dim': dim, 'name': 'wf_int', 'pre_mesh': pre_mesh_int, 'fem_order': fem_order}
partial_args_dict_ext = {'dim': dim, 'name': 'wf_ext', 'pre_mesh': pre_mesh_ext, 'fem_order': fem_order}
partial_args_dict_ext['pre_ebc_dict'] = {('vertex', 0): 0.0,}
region_key_int = ('facet', 201)
region_key_ext = ('facet', 200)
poisson.set_wf_int(partial_args_dict_int, {('subomega', 300): 1.0})
poisson.set_wf_ext(partial_args_dict_ext, density=None)
poisson.set_default_solver(region_key_int=region_key_int,
                           region_key_ext=region_key_ext)

In [4]:
# Solving
solver = poisson.default_solver
solver.solve(use_reduced_mtx_vec=True)

In [5]:
# Comparison against the analytical solution
from femtoscope.misc.analytical import potential_ellipsoid
import warnings
warnings.filterwarnings('ignore')

coors_int = solver.wf_int.field.coors
coors_int[:, 0][np.where(coors_int[:, 0] < 0)[0]] = 0
sol_fem_int = solver.sol_int
sol_ana_int = potential_ellipsoid(coors_int, sa, 1.0, sc=sc, rho=1.0)
err_rel_int = abs(sol_fem_int - sol_ana_int) / abs(sol_ana_int)
print(err_rel_int)

[0.00051354 0.00069802 0.00051332 ... 0.0002993  0.00029934 0.00024164]


## Chameleon problem on unbounded domain made easy

Here, we compute the chameleon field sourced by the same flat ellipsoid

In [ ]:
# parameters
alpha = 0.1
npot = 2
rho_min = 1.0
rho_max = 1e2
phi_min = rho_max ** (-1 / (npot + 1))
phi_max = rho_min ** (-1 / (npot + 1))

# Creation and setting of the 'Chameleon' object
chameleon = Chameleon({'alpha': alpha, 'npot': npot}, dim, Rc=Rc, coorsys=coorsys)
partial_args_dict_ext['pre_ebc_dict'][('vertex', 0)] = phi_max
chameleon.set_wf_int(partial_args_dict_int, {('subomega', 300): rho_max, ('subomega', 301): rho_min})
chameleon.set_wf_residual(partial_args_dict_int, {('subomega', 300): rho_max, ('subomega', 301): rho_min})
chameleon.set_wf_ext(partial_args_dict_ext, density=rho_min)
chameleon.set_default_solver()
chameleon.set_default_monitor(10)

In [ ]:
# Removing meshfiles
from pathlib import Path
from femtoscope import MESH_DIR
Path(MESH_DIR / meshint_name).unlink()
Path(MESH_DIR / meshext_name).unlink()